In [ ]:
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/Maestría/Investigación/Revisión de Literatura/Datasets Latinamerica/Raw Search/
!ls

/content/drive/MyDrive/Maestría/Investigación/Revisión de Literatura/Datasets Latinamerica
compare_datasets.ipynb		Scopus_original.txt
data_preprocessing_summary.csv	Scopus_WoS_preprocessed.csv
Merge_Scopus_Wos_Pubmed.ipynb	scopus_wos_pubmed_initial.csv
Pubmed.csv			WoS_original.txt


In [ ]:
scopus_wos = pd.read_csv('Scopus_WoS_preprocessed.csv')
scopus_wos.head(3)

,Authors,Title,Year,Source title,Volume,Issue,Art. No.,Page start,Page end,Page count,...,Document Type,Source,Subject,EID,duplicatedIn,country,emailHost,institution,institutionWithCountry,authorFull
0,"Serrano-Brambila E.A., Moreno-Alcazar O.M., Ne...",[Clinical practice guideline. Traumatic urethr...,2013,Revista médica del Instituto Mexicano del Segu...,51,4,NaN,472,479,7.0,...,Article,Scopus,NaN,2-s2.0-84887473707,NaN,Mexico,gmail.com,NaN,NaN,Serrano-Brambila E.A.; Moreno-Alcazar O.M.; Ne...
1,"Cuoghi I.C., da Silva Soares M.F., dos Santos ...",10-year opportunistic mammographic screening s...,2022,Journal of Global Health,12,NaN,04061,NaN,NaN,NaN,...,Article,Scopus,NaN,2-s2.0-85139880822,NaN,Brazil,usp.br,NaN,NaN,Cuoghi I.C.; da Silva Soares M.F.; dos Santos ...
2,"Balducci, T., Rasgado-Toledo, J., Valencia, A....",A behavioral and brain imaging dataset with fo...,2022,SCIENTIFIC DATA,9,1,581,NaN,NaN,10.0,...,Article; Data Paper,WoS,Science & Technology - Other Topics,WOS:000857064600002,NaN,Mexico;Netherlands;China,comunidad.unam.mx,Univ Nacl Autonoma Mexico;Univ Groningen;Inst ...,"Univ Nacl Autonoma Mexico, Mexico;Univ Groning...","Balducci, Thania; Rasgado-Toledo, Jalil; Valen..."


In [ ]:
print(f'Number of papers: {scopus_wos.shape[0]}')

Number of papers: 419


### Pubmed

In [ ]:
pubmed = pd.read_csv('Pubmed.csv')
pubmed.rename(columns={'PMID':'PubMed ID', 'Journal/Book':'Source title', 'Publication Year':'Year'}, inplace=True)
pubmed = pubmed[~pubmed['DOI'].isna()]
pubmed = pubmed[['PubMed ID', 'Title', 'Authors', 'Source title', 'Year', 'PMCID', 'DOI']]
pubmed.head(3)

,PubMed ID,Title,Authors,Source title,Year,PMCID,DOI
0,32022198,Storage and disposal of pharmaceuticals at hom...,"Constantino VM, Fregonesi BM, Tonani KAA, Zagu...",Cien Saude Colet,2020,NaN,10.1590/1413-81232020252.10882018
1,34566452,A database and checklist of geometrid moths (L...,"Murillo-Ramos L, Sihvonen P, Brehm G, Ríos-Mal...",Biodivers Data J,2021,PMC8433126,10.3897/BDJ.9.e68693
2,32983559,Towards a unified Latin American Natural Produ...,L Medina-Franco J.,Future Sci OA,2020,PMC7491008,10.2144/fsoa-2020-0068


In [ ]:
print(f'Number of papers: {pubmed.shape[0]}')

Number of papers: 137


### Find duplicates

In [ ]:
def check_doi(list1, list2, name1, name2):
  dois_in1_not_in2 = list(set(list1) - set(list2))

  dois_in2_not_in1 = list(set(list2) - set(list1))

  if (len(dois_in1_not_in2) == 0) and (len(dois_in2_not_in1) == 0):
    print('Both files have the same dois!')
  else:
    if (len(dois_in1_not_in2) != 0):
      print(f'{name1} has {len(dois_in1_not_in2)} dois that are not in {name2}')
      #print(f'{name1} has the dois: {dois_in1_not_in2} that are not in {name2}')
    if (len(dois_in2_not_in1) != 0):
      print(f'{name2} has {len(dois_in2_not_in1)} dois that are not in {name1}')
      #print(f'{name2} has the dois: {dois_in2_not_in1} that are not in {name1}')
  return dois_in1_not_in2, dois_in2_not_in1

In [ ]:
list1 = scopus_wos['DOI'].tolist()

list2 = pubmed['DOI'].tolist()

dois_scopus, dois_pubmed = check_doi(list1, list2, name1='Scopus + WoS', name2='Pubmed')

Scopus + WoS has 341 dois that are not in Pubmed
Pubmed has 88 dois that are not in Scopus + WoS


### Merge the files

In [ ]:
pubmed = pubmed[pubmed['DOI'].isin(dois_pubmed)]

In [ ]:
# Concatenate vertically
result = pd.concat([scopus_wos, pubmed], axis=0)
result

,Authors,Title,Year,Source title,Volume,Issue,Art. No.,Page start,Page end,Page count,...,Source,Subject,EID,duplicatedIn,country,emailHost,institution,institutionWithCountry,authorFull,PMCID
0,"Serrano-Brambila E.A., Moreno-Alcazar O.M., Ne...",[Clinical practice guideline. Traumatic urethr...,2013,Revista médica del Instituto Mexicano del Segu...,51,4,NaN,472,479,7.0,...,Scopus,NaN,2-s2.0-84887473707,NaN,Mexico,gmail.com,NaN,NaN,Serrano-Brambila E.A.; Moreno-Alcazar O.M.; Ne...,NaN
1,"Cuoghi I.C., da Silva Soares M.F., dos Santos ...",10-year opportunistic mammographic screening s...,2022,Journal of Global Health,12,NaN,04061,NaN,NaN,NaN,...,Scopus,NaN,2-s2.0-85139880822,NaN,Brazil,usp.br,NaN,NaN,Cuoghi I.C.; da Silva Soares M.F.; dos Santos ...,NaN
2,"Balducci, T., Rasgado-Toledo, J., Valencia, A....",A behavioral and brain imaging dataset with fo...,2022,SCIENTIFIC DATA,9,1,581,NaN,NaN,10.0,...,WoS,Science & Technology - Other Topics,WOS:000857064600002,NaN,Mexico;Netherlands;China,comunidad.unam.mx,Univ Nacl Autonoma Mexico;Univ Groningen;Inst ...,"Univ Nacl Autonoma Mexico, Mexico;Univ Groning...","Balducci, Thania; Rasgado-Toledo, Jalil; Valen...",NaN
3,"Krauskopf, E.",A bibiliometric analysis of the Journal of Inf...,2018,JOURNAL OF INFECTION AND PUBLIC HEALTH,11,2,NaN,224,229,6.0,...,WoS,"Public, Environmental & Occupational Health; I...",WOS:000426961500014,NaN,Chile,unab.cl,Univ Andres Bello,"Univ Andres Bello, Chile","Krauskopf, Erwin",NaN
4,"Sevillano-Jimenez J., Carrion-Chambilla M., Es...",A bibliometric analysis of 47-years of researc...,2023,Electronic Journal of General Medicine,20,4,em488,NaN,NaN,NaN,...,Scopus,NaN,2-s2.0-85152736068,NaN,Peru,upch.pe,NaN,NaN,Sevillano-Jimenez J.; Carrion-Chambilla M.; Es...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,"Girard P, Zihindula MC, Delinois D, Kolontchan...",[Nailing of long bone fractures in precarious ...,2021,Pan Afr Med J,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PMC8418173
133,"Pascual A, Tupinambá-Simões F, Guerra-Hernánde...",High-resolution planet satellite imagery and m...,2022,J Environ Manage,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
135,"Magdalena UR, da Silva LAE, Oliveira FA, Lima ...",Geographic review on the specimens of the Caat...,2019,Biodivers Data J,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PMC6787104
136,"Ostermann T, Zillmann H, Raak CK, Buessing A, ...",CAMbase--a XML-based bibliographical database ...,2007,Biomed Digit Libr,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PMC1853104


In [ ]:
result.to_csv('scopus_wos_pubmed_initial.csv')